In [135]:
import os
import sys
import pandas as pd
import googletrans
from googletrans import Translator
%matplotlib inline
import matplotlib.pyplot as plt

In [136]:
ROOT_PATH = os.path.abspath(os.getcwd())
CALPI = "calpi"
SETTELMENT = "settelments"



def get_data(agg_type, num):
    """returns dataframe of the requested csv
    
    parameters
    ----------
    agg_type - aggregation type of the csv needed
    num - election number
    
    return
    ------
    DataFrame
    """
    if type(num) is not str:
        num = str(num)
    
    
    path = os.path.join(ROOT_PATH,num+agg_type+".xlsx")
    return pd.read_excel(path,dtype=object)    

In [137]:
def getRideofColm(df):
    party_dict = {}    

    for column in df.columns[7:]:
            party_dict[column]=df[column].sum()
    for c in party_dict.keys():
        if party_dict[c] < 135720:
               df = df.drop(c,axis=1)
                
    for i,r in df.iterrows():
        for c in df.columns[7:]:
            x = r[c]/r['valid votes']
            df.at[i , c]=x

    return df

In [140]:
df = get_data(SETTELMENT,23)
df = getRideofColm(df)
df

TypeError: unsupported operand type(s) for +: 'int' and 'str'